<a href="https://colab.research.google.com/github/Dipendra7/2015lab1/blob/master/pytorch001_SNN_Norse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install atari pong enviroment

In [17]:
!pip install "gym[atari, accept-rom-license]"

#Get all the includes

In [18]:
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions as distributions
import matplotlib.pyplot as plt
import numpy as np
import gym

#Video imports
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.color import rgb2gray
from skimage.transform import resize

#timer
import signal

print(gym.__version__)

0.25.2


Install norse package

In [19]:
!pip install norse


In [20]:
import torch
import torch.nn as nn
import norse.torch as snn
import torch.nn.functional as F

# Create SNN Arhitecture that will inherit layers from nn.Module
class SNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout=0.5):
        super(SNN, self).__init__()

        # Convolutional layer
        self.fc0 = nn.Conv2d(3, 1, 5)

        # First fully connected layer
        self.fc2 = nn.Linear(32136, 256)  # Adjusted based on input dim

        # Output layer
        self.fc3 = nn.Linear(256, output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

        # LIF cells to replace ReLU activations
        self.lif0 = snn.LIFCell()  # For the first convolutional layer
        self.lif1 = snn.LIFCell()  # For the first fully connected layer

    def forward(self, x):
        # Change input shape to match the convolutional layer expectations
        x = x.permute(0, 3, 1, 2)

        # Convolutional layer + LIF
        x = self.fc0(x)
        lif0_state = None
        x, lif0_state = self.lif0(x, lif0_state)

        # Flatten the output for the fully connected layer
        x = torch.flatten(x, start_dim=1)  # Flatten along the batch dimension

        # First fully connected layer + LIF
        x = self.fc2(x)
        lif1_state = None
        x, lif1_state = self.lif1(x, lif1_state)

        # Apply dropout
        x = self.dropout(x)

        # Output layer (no activation for final output)
        x = self.fc3(x)

        return x


Weights initialization
Currently Xavier

In [21]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight)
        m.bias.data.fill_(0)

Train Loop

In [22]:
def train(env, policy, optimizer, discount_factor, frames):

    policy.train()

    log_prob_actions = []
    rewards = []
    done = False
    episode_reward = 0

    state = env.reset()

    while not done:

        #state = torch.FloatTensor(state).unsqueeze(0).to('cuda')
        state = torch.FloatTensor(state).unsqueeze(0).to(device)

        action_pred = policy(state)

        action_prob = F.softmax(action_pred, dim = -1)

        dist = distributions.Categorical(action_prob)

        action = dist.sample()

        log_prob_action = dist.log_prob(action)

        state, reward, done, _ = env.step(action.item())
        frames.append(state)
        env.render('rgb_array')

        log_prob_actions.append(log_prob_action.reshape(1))
        rewards.append(reward)

        episode_reward += reward

    log_prob_actions = torch.cat(log_prob_actions, dim=0)

    returns = calculate_returns(rewards, discount_factor)

    loss = update_policy(returns, log_prob_actions, optimizer)

    return loss, episode_reward

Function to measure results

In [23]:
def calculate_returns(rewards, discount_factor, normalize = True):

    returns = []
    R = 0

    for r in reversed(rewards):
        R = r + R * discount_factor
        returns.insert(0, R)

    returns = torch.tensor(returns)

    if normalize:
        returns = (returns - returns.mean()) / returns.std()

    return returns

Evaluation Loop

In [24]:
def evaluate(env, policy):

    policy.eval()

    done = False
    episode_reward = 0

    state = env.reset()

    while not done:

        state = torch.FloatTensor(state).unsqueeze(0)
        state = state.to(next(policy.parameters()).device)

        with torch.no_grad():

            action_pred = policy(state)

            action_prob = F.softmax(action_pred, dim = -1)

        action = torch.argmax(action_prob, dim = -1)

        state, reward, done, _ = env.step(action.item())

        episode_reward += reward

    return episode_reward

In [25]:
def update_policy(returns, log_prob_actions, optimizer):

    # Ensure both tensors are on the same device
    device = log_prob_actions.device  # Get the device of log_prob_actions
    returns = returns.to(device)  # Move returns to the same device

    returns = returns.detach()

    loss = - (returns * log_prob_actions).sum()

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    return loss.item()

Function to save video and gifs

In [26]:
def save_video(frames, filename):
   height, width, _ = frames[0].shape
   fig = plt.figure()
   plt.axis('off')
   ims = [[plt.imshow(frame, animated=True)] for frame in frames]

   ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)
   ani.save(filename, writer='pillow')
   print(f"Video saved as {filename}")

In [27]:
class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException

signal.signal(signal.SIGALRM, timeout_handler)

<function __main__.timeout_handler(signum, frame)>

#Setup

In [28]:
SEED = 1234

train_env = gym.make('Pong-v0')
test_env = gym.make('Pong-v0')
train_env.metadata['render_fps'] = 20

train_env.seed(SEED);
test_env.seed(SEED+1);
np.random.seed(SEED);
torch.manual_seed(SEED);

state = train_env.reset()

print('state type:', type(state))
print('state shape:', state.shape)

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


state type: <class 'numpy.ndarray'>
state shape: (210, 160, 3)


In [29]:
# if GPU is to be used
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)

INPUT_DIM = train_env.observation_space.shape[0]
HIDDEN_DIM = 1024
OUTPUT_DIM = train_env.action_space.n

#policy = MLP(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM)
policy=SNN(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
#policy = DQN(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
policy.apply(init_weights)

#LEARNING_RATE = 0.001
LEARNING_RATE = 0.00002  # Lower the learning rate
optimizer = optim.Adam(policy.parameters(), lr = LEARNING_RATE)

MAX_EPISODES = 500
DISCOUNT_FACTOR = 0.99
TIME = 600

#Learning process

In [30]:
def trainSNN(MAX_EPISODES, DISCOUNT_FACTOR):
  for episode in range(1, MAX_EPISODES+1):
    frames = []
    loss, train_reward = train(train_env, policy, optimizer, DISCOUNT_FACTOR, frames)
    test_reward = evaluate(test_env, policy)
    save_video(frames, f'episode_{episode + 1}.gif')
    print(f'| Episode: {episode:3} | Train Reward: {train_reward:5.1f} | Test Reward: {test_reward:5.1f} |')

In [32]:
def trainSNN(MAX_EPISODES, DISCOUNT_FACTOR):
    for episode in range(1, MAX_EPISODES + 1):
        print(f'Starting Episode {episode}/{MAX_EPISODES}')  # Diagnostic print statement

        frames = []
        try:
            loss, train_reward = train(train_env, policy, optimizer, DISCOUNT_FACTOR, frames)
        except Exception as e:
            print(f"Error during training in episode {episode}: {e}")
            continue  # Skip to the next episode in case of an error

        try:
            test_reward = evaluate(test_env, policy)
        except Exception as e:
            print(f"Error during evaluation in episode {episode}: {e}")
            test_reward = float('-inf')  # Assign a very low reward if evaluation fails

        #try:
            #save_video(frames, f'episode_{episode}.gif')
        #except Exception as e:
            #print(f"Error saving video for episode {episode}: {e}")

        print(f'| Episode: {episode:3} | Train Reward: {train_reward:5.1f} | Test Reward: {test_reward:5.1f} |')

    print(f'Training completed for {MAX_EPISODES} episodes.')

# Example of how you would call this function:
trainSNN(MAX_EPISODES=1000, DISCOUNT_FACTOR=0.99)


Starting Episode 1/1000
| Episode:   1 | Train Reward: -21.0 | Test Reward: -21.0 |
Starting Episode 2/1000


/usr/local/lib/python3.10/dist-packages/matplotlib/animation.py:884: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you output the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(


| Episode:   2 | Train Reward: -19.0 | Test Reward: -21.0 |
Starting Episode 3/1000
| Episode:   3 | Train Reward: -20.0 | Test Reward: -21.0 |
Starting Episode 4/1000
| Episode:   4 | Train Reward: -20.0 | Test Reward: -21.0 |
Starting Episode 5/1000
| Episode:   5 | Train Reward: -21.0 | Test Reward: -21.0 |
Starting Episode 6/1000
| Episode:   6 | Train Reward: -20.0 | Test Reward: -21.0 |
Starting Episode 7/1000
| Episode:   7 | Train Reward: -21.0 | Test Reward: -21.0 |
Starting Episode 8/1000
| Episode:   8 | Train Reward: -20.0 | Test Reward: -21.0 |
Starting Episode 9/1000
| Episode:   9 | Train Reward: -21.0 | Test Reward: -21.0 |
Starting Episode 10/1000
| Episode:  10 | Train Reward: -20.0 | Test Reward: -21.0 |
Starting Episode 11/1000
| Episode:  11 | Train Reward: -21.0 | Test Reward: -21.0 |
Starting Episode 12/1000
| Episode:  12 | Train Reward: -20.0 | Test Reward: -21.0 |
Starting Episode 13/1000
| Episode:  13 | Train Reward: -21.0 | Test Reward: -21.0 |
Starting Epi

In [33]:
!pip install optuna
import optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.5 MB/s eta 0:00:00


In [34]:
def objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    hidden_dim = trial.suggest_int("hidden_dim", 256, 1024)
    dropout = trial.suggest_float("dropout", 0.1, 0.8)

    # Initialize model, optimizer with suggested hyperparameters
    policy = SNN(INPUT_DIM, hidden_dim, OUTPUT_DIM, dropout).to(device)
    policy.apply(init_weights)
    optimizer = optim.Adam(policy.parameters(), lr=learning_rate)

    # Training loop (simplified for brevity)
    for episode in range(1, MAX_EPISODES + 1):
        loss, train_reward = train(train_env, policy, optimizer, DISCOUNT_FACTOR, [])

        # Evaluate on validation set - Replace this with your actual validation logic
        validation_reward = evaluate(test_env, policy) # Example: Assuming you have a validation_env

        trial.report(validation_reward, episode)

        if trial.should_prune():
            raise optuna.TrialPruned()

    return validation_reward

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
study = optuna.create_study(direction="maximize")  # Maximize validation reward
study.optimize(objective, n_trials=500)  # Run 100 trials

[I 2024-09-03 03:26:11,638] A new study created in memory with name: no-name-fda8b776-8b3a-4bfe-a1f2-cbff5e157d32
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
[I 2024-09-03 05:24:51,951] Trial 0 finished with value: -21.0 and parameters: {'learning_rate': 0.007877002590862171, 'hidden_dim': 398, 'dropout': 0.3646383247633094}. Best is trial 0 with value: -21.0.
[I 2024-09-03 07:42:13,714] Trial 1 finished with value: -21.0 and parameters: {'learning_rate': 0.0007300756304241345, 'hidden_dim': 670, 'dropout': 0.5404027133938042}. Best is trial 0 with value: -21.0.
[I 2024-09-03 10:00:28,148] Trial 2 finished with value: -21.0 and parameters: {'learning_rate': 0.0002338893667880556, 'hidden_dim': 1010, 'dropout': 0.433656019032452}. Best is trial 0 with value: 

In [ ]:
!pip freeze > requirements.txt

In [ ]:
signal.alarm(TIME)

try:
    trainSNN(MAX_EPISODES, DISCOUNT_FACTOR)
except TimeoutException:
    signal.alarm(0)
    test_reward = evaluate(test_env, policy)
    print(f'| Final test. Reward: {test_reward:5.1f} |')



In [ ]:
# signal.alarm(TIME)

# try:
#     trainDQN(MAX_EPISODES, DISCOUNT_FACTOR)
# except TimeoutException:
#     signal.alarm(0)
#     test_reward = evaluate(test_env, policy)
#     print(f'| Final test. Reward: {test_reward:5.1f} |')

